In [1]:
import Utils
from DataLoader import *
from FeatureCreator import *
from Model import *
from CouponCreator import CouponCreator

In [2]:
config = Utils.read_json('../config.json')
Utils.print_json(config)

{
    "data": {
        "path": "../data/",
        "files": {
            "baskets": "baskets.parquet",
            "coupons": "coupons.parquet",
            "coupons_index": "coupon_index.parquet"
        }
    },
    "model": {
        "test_week": 90,
        "n_shoppers": 2000,
        "n_products": 250,
        "train_window": 4,
        "trend_windows": [
            1,
            3,
            5
        ],
        "n_coupons": 5,
        "discounts": [
            0.15,
            0.2,
            0.25,
            0.3
        ]
    }
}


In [3]:
# DATA LOADING
dataloader = DataLoader(config)
dataset = dataloader.get_dataset()

Read dataset.parquet.gzip from cache...
dataset.parquet.gzip was not found in cache.
Executing callback function "create_dataset" to create dataset.parquet.gzip
57.5% memory reduction for baskets.parquet (from 2626.10 MB to 1116.09 MB)
60.0% memory reduction for coupons.parquet (from 1716.61 MB to 686.65 MB)
62.5% memory reduction for coupon_index.parquet (from 0.31 MB to 0.11 MB)
replaced missing prices with mode
Wrote dataset.parquet.gzip to ../cache/
Successfully read dataset.parquet.gzip into memory.


In [4]:
dataset

,week,shopper,product,price,discount,purchased,week_hist
0,86,0,0,688.0,0.0,0.0,NaN
1,86,0,1,560.0,0.0,0.0,NaN
2,86,0,2,773.0,0.0,0.0,NaN
3,86,0,3,722.0,0.0,0.0,NaN
4,86,0,4,620.0,0.0,0.0,"[24, 40, 45, 51, 54, 58, 60, 70]"
...,...,...,...,...,...,...,...
2499995,90,1999,245,549.0,0.0,0.0,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89]"
2499996,90,1999,246,702.0,0.0,0.0,NaN
2499997,90,1999,247,670.0,0.0,0.0,NaN
2499998,90,1999,248,490.0,0.0,0.0,NaN


In [5]:
# FEATURE CREATION
feature_creator = FeatureCreator(dataset, config)
model_data = feature_creator.get_model_data()

Read model_data.parquet.gzip from cache...
model_data.parquet.gzip was not found in cache.
Executing callback function "create_features" to create model_data.parquet.gzip
Wrote model_data.parquet.gzip to ../cache/
Successfully read model_data.parquet.gzip into memory.


In [6]:
# MODELING - Train-Test-Split
model = Model(model_data)
X_train, y_train, X_test, y_test = model.train_test_split(config)

/Users/leonhagel/Documents/coding/github/mlim-g2/env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/leonhagel/Documents/coding/github/mlim-g2/env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [7]:
X_test

,price,discount,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
2000000,688.0,0.0,0.0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-0.595444
2000001,560.0,0.0,0.0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,0.615436
2000002,773.0,0.0,0.0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-0.713309
2000003,722.0,0.0,0.0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-1.397781
2000004,620.0,0.0,0.0,0,1,0,20.0,0.0,0.000000,0.0,0.088889,0.074996,-0.163894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,549.0,0.0,0.0,0,0,1,1.0,1.0,0.333333,0.2,0.133333,0.248261,-0.818372
2499996,702.0,0.0,0.0,0,0,1,104.0,0.0,0.000000,0.0,0.000000,0.248261,-0.305680
2499997,670.0,0.0,0.0,0,0,1,104.0,0.0,0.000000,0.0,0.000000,0.248261,-1.574931
2499998,490.0,0.0,0.0,0,0,1,104.0,0.0,0.000000,0.0,0.000000,0.248261,0.792432


In [8]:
# norm:                       0.07600950467011207
# elast:                      0.07609908479679367
# discount_buy:               0.07601130911565658
# costumer_redemption_rate:   0.07607927210850536
# redemption_rate :           0.07670198091212563

In [9]:
# FITTING THE MODEL
model.fit(X_train, y_train)
y_hat = model.predict(X_train)
log_loss_score = model.log_loss_score(y_train, y_hat)
log_loss_score

0.07610826369684603

In [10]:
coupon_creator = CouponCreator(model)

In [11]:
X_template = X_test.copy()
X_template['discount'] = None
X_template['substitue_discount'] = 0
X_template

,price,discount,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
2000000,688.0,None,0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-0.595444
2000001,560.0,None,0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,0.615436
2000002,773.0,None,0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-0.713309
2000003,722.0,None,0,0,1,0,104.0,0.0,0.000000,0.0,0.000000,0.074996,-1.397781
2000004,620.0,None,0,0,1,0,20.0,0.0,0.000000,0.0,0.088889,0.074996,-0.163894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,549.0,None,0,0,0,1,1.0,1.0,0.333333,0.2,0.133333,0.248261,-0.818372
2499996,702.0,None,0,0,0,1,104.0,0.0,0.000000,0.0,0.000000,0.248261,-0.305680
2499997,670.0,None,0,0,0,1,104.0,0.0,0.000000,0.0,0.000000,0.248261,-1.574931
2499998,490.0,None,0,0,0,1,104.0,0.0,0.000000,0.0,0.000000,0.248261,0.792432


In [12]:
discounts = [.15, .2, .25, .3]
coupon_creator.run(discounts, X_template)

,shopper,week,coupon,product,discount
2000171,0,90,0,171,0.15
2000076,0,90,1,76,0.15
2000067,0,90,2,67,0.15
2000157,0,90,3,157,0.15
2000211,0,90,4,211,0.30
...,...,...,...,...,...
2062339,249,90,0,89,0.15
2062411,249,90,1,161,0.25
2062313,249,90,2,63,0.30
2062262,249,90,3,12,0.15
